In [ ]:
# First, install the necessary library
!pip install youtube_transcript_api


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 11.1 MB/s eta 0:00:00


### METODE 1: YOUTUBE TRANSKRIP API TO TXT

In [ ]:
import whisper
import os
import re
from pytube import YouTube
import datetime

def get_video_id(url):
    """Extract video ID from YouTube URL"""
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None

def download_youtube_audio(url, output_path="audio"):
    """Download audio from YouTube video"""
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        # Create YouTube object
        yt = YouTube(url)
        
        # Get video details for file naming
        video_id = get_video_id(url)
        video_title = yt.title
        safe_title = re.sub(r'[^\w\-_]', '_', video_title)
        
        print(f"Downloading audio from: {video_title}")
        
        # Get audio stream and download
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(output_path=output_path)
        
        # Rename to mp3
        base, ext = os.path.splitext(output_file)
        new_file = f"{base}.mp3"
        os.rename(output_file, new_file)
        
        print(f"Audio downloaded to: {new_file}")
        return new_file, safe_title
        
    except Exception as e:
        print(f"Error downloading YouTube audio: {e}")
        return None, None

def transcribe_audio(audio_path, language="id"):
    """Transcribe audio using OpenAI Whisper"""
    try:
        print(f"Loading Whisper model...")
        model = whisper.load_model("medium")  # You can change model size: tiny, base, small, medium, large
        
        print(f"Transcribing audio... (this may take some time)")
        result = model.transcribe(audio_path, language=language)
        
        return result["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Main process
def process_youtube_speech(youtube_url, language="id"):
    """Process YouTube video: download audio and transcribe"""
    # Download audio
    audio_file, video_title = download_youtube_audio(youtube_url)
    
    if not audio_file:
        return
    
    # Transcribe audio
    transcription = transcribe_audio(audio_file, language=language)
    
    if not transcription:
        return
    
    # Save transcription to file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"Soekarno_{timestamp}.txt"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(f"Source: {youtube_url}\n")
        f.write(f"Title: {video_title}\n")
        f.write(f"Transcribed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(transcription)
    
    print(f"✓ Transcription complete! Saved to {output_filename}")
    print("\nPreview:")
    print(transcription[:500] + "..." if len(transcription) > 500 else transcription)
    
    return output_filename

# Example usage
youtube_url = "https://www.youtube.com/watch?v=N_Cp4KBrRZw"  # Change to your Soekarno speech video
process_youtube_speech(youtube_url, language="id")

Created output directory
Transcript:
[Music] The purpose of the United Nations should be to solve problems, to use it as a mere debating platform or as a propaganda outlet or as an extension of domestic politics is to pervert the high ideals which should imbue this body in order to modernize in order to modernize and make efficient our organization. Perhaps even the secretariat under the leadership of its secretary general. may need to be revised. In saying this, I am not most definitely not in any way criticizing or denouncing the present secretary general who is striving to do a good job under outmoded conditions which must at times seem impossible. How then can they be efficient? How can members of those two groups in the world, groups which are a reality and which must be accepted, how can members of those two groups feel at ease in this organization and have the necessary utmost confidence in it. Since the war, we have witnessed three great permanent phenomena. First is the rise o

### METHOD 2: OPEN AI WHISPER

In [1]:
# Install all required packages
!pip install openai-whisper ffmpeg-python pytube

In [3]:
!pip install yt-dlp

   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ------------------------- -------------- 2.1/3.3 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 9.6 MB/s eta 0:00:00


In [4]:
# Option 1: Try to install FFmpeg via pip (simpler but may not always work)
!pip install ffmpeg-python

# Option 2: Try to install via conda if you're using Anaconda
# !conda install -c conda-forge ffmpeg

In [5]:
import whisper
import os
import re
from pytube import YouTube
import datetime
import yt_dlp

def get_video_id(url):
    """Extract video ID from YouTube URL"""
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None


def download_with_ytdlp(url, output_path="audio", ffmpeg_location=None):
    """Alternative download using yt-dlp with ffmpeg handling"""
    try:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        video_id = get_video_id(url)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{output_path}/audio_{video_id}_{timestamp}.mp3"
        
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_file,
            'quiet': False,
            'no_warnings': False
        }
        
        # Add ffmpeg location if provided
        if ffmpeg_location:
            ydl_opts['ffmpeg_location'] = ffmpeg_location
            ydl_opts['postprocessors'] = [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }]
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            video_title = info.get('title', f"video_{video_id}")
            
        print(f"Audio downloaded to: {output_file}")
        return output_file, video_title
        
    except Exception as e:
        print(f"Error downloading with yt-dlp: {e}")
        return None, None

def download_youtube_audio(url, output_path="audio"):
    """Download audio from YouTube video using pytube"""
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        # Create YouTube object
        yt = YouTube(url)
        
        # Get video details for file naming
        video_id = get_video_id(url)
        video_title = yt.title
        safe_title = re.sub(r'[^\w\-_]', '_', video_title)
        
        print(f"Downloading audio from: {video_title}")
        
        # Get audio stream and download
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(output_path=output_path)
        
        # Rename to mp3
        base, ext = os.path.splitext(output_file)
        new_file = f"{base}.mp3"
        os.rename(output_file, new_file)
        
        print(f"Audio downloaded to: {new_file}")
        return new_file, safe_title
        
    except Exception as e:
        print(f"Error with pytube: {e}")
        return None, None
def transcribe_audio(audio_path, language="id"):
    """Transcribe audio using OpenAI Whisper"""
    try:
        print(f"Loading Whisper model...")
        model = whisper.load_model("medium")  # You can change model size: tiny, base, small, medium, large
        
        print(f"Transcribing audio... (this may take some time)")
        result = model.transcribe(audio_path, language=language)
        
        return result["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def process_youtube_speech(youtube_url, language="id"):
    """Process YouTube video: download audio and transcribe"""
    # Try downloading with pytube first
    audio_file, video_title = download_youtube_audio(youtube_url)
    
    # If pytube fails, try yt-dlp
    if not audio_file:
        audio_file, video_title = download_with_ytdlp(youtube_url)
    
    if not audio_file:
        print("Failed to download audio using both methods.")
        return
    
    # Rest of the function remains the same
    # Transcribe audio
    transcription = transcribe_audio(audio_file, language=language)
    
    if not transcription:
        return
    
    # Save transcription to file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"Soekarno_{timestamp}.txt"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(f"Source: {youtube_url}\n")
        f.write(f"Title: {video_title}\n")
        f.write(f"Transcribed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(transcription)
    
    print(f"✓ Transcription complete! Saved to {output_filename}")
    print("\nPreview:")
    print(transcription[:500] + "..." if len(transcription) > 500 else transcription)
    
    return output_filename

# Example usage
youtube_url = "https://www.youtube.com/watch?v=bcIk9n6nRUo&ab_channel=HendriTeja"  # Change to your Soekarno speech video
process_youtube_speech(youtube_url, language="id")

Error with pytube: HTTP Error 400: Bad Request
[youtube] Extracting URL: https://www.youtube.com/watch?v=bcIk9n6nRUo&ab_channel=HendriTeja
[youtube] bcIk9n6nRUo: Downloading webpage
[youtube] bcIk9n6nRUo: Downloading tv client config
[youtube] bcIk9n6nRUo: Downloading tv player API JSON
[youtube] bcIk9n6nRUo: Downloading ios player API JSON
[youtube] bcIk9n6nRUo: Downloading m3u8 information
[info] bcIk9n6nRUo: Downloading 1 format(s): 251
[download] Destination: audio\audio_bcIk9n6nRUo_20250523_220126.mp3
[download] 100% of    2.25MiB in 00:00:00 at 4.43MiB/s   
Audio downloaded to: audio/audio_bcIk9n6nRUo_20250523_220126.mp3
Loading Whisper model...


 20%|███████▋                              | 294M/1.42G [00:43<02:54, 7.01MiB/s]


KeyboardInterrupt: 

### Scrapping from Website

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# URL target
url = "https://www.goodreads.com/author/quotes/661589.Sukarno"

# User-Agent header untuk meniru browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

quotes_list = []

while True:
    # Mengirim permintaan HTTP
    response = requests.get(url, headers=headers)
    
    # Memeriksa status response
    if response.status_code != 200:
        print(f"Gagal mengakses halaman. Kode status: {response.status_code}")
        break
        
    # Parsing konten HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Mencari semua elemen kutipan
    quotes = soup.find_all('div', class_='quoteText')
    
    # Mengekstrak teks kutipan
    for quote in quotes:
        text = quote.get_text(strip=True).split('―')[0]
        text = text.replace('“', '').replace('”', '').strip()
        quotes_list.append(text)
    
    # Mencari link halaman berikutnya
    next_page = soup.find('a', class_='next_page')
    
    if not next_page:
        break  # Tidak ada halaman berikutnya
        
    url = "https://www.goodreads.com" + next_page['href']
    sleep(2)  # Jeda untuk menghormati server

# Membuat DataFrame
df = pd.DataFrame(quotes_list, columns=['Quotes'])

# Menyimpan ke CSV
df.to_csv('quotes_soekarno.csv', index=False)

# Menampilkan hasil
print(f"Berhasil mengumpulkan {len(df)} kutipan:")
df.head()

Berhasil mengumpulkan 25 kutipan:


,Quotes
0,"Kami menggoyangkan langit, menggempakan darat,..."
1,"This country, the Republic of Indonesia, does ..."
2,"Bebek berjalan berbondong-bondong, akan tetapi..."
3,I hate imperialism. I detest colonialism. And ...
4,"Learning without thinking is useless, but thin..."


In [ ]:
import pandas as pd
import json
import os
import re

base_path = r"c:\PythonVSCenv\Capstone\scrapping\output"
hatta_csv = os.path.join(base_path, "quotes_hatta.csv")
soekarno_csv = os.path.join(base_path, "quotes_soekarno.csv")
output_json = os.path.join(base_path, "content_author_quotes.json")

def load_quotes(path):
    if not os.path.exists(path):
        return []
    df = pd.read_csv(path)
    col = 'Quotes' if 'Quotes' in df.columns else df.columns[0]
    return [q.strip().strip('"') for q in df[col].dropna().astype(str) if q.strip()]

tags = [
    'greeting','whoami','nationalism','revolution','independence',
    'unity','advice','international','struggle','goodbye'
]

keywords = {
    'greeting': ['selamat pagi','halo','hai','salam'],
    'whoami': ['aku adalah','saya adalah','identitas saya'],
    'nationalism': ['bangsa','negara','nasional','tanah air'],
    'revolution': ['revolusi','letusan','mengguncang','gempakan'],
    'independence': ['merdeka','kemerdekaan','bebas','penjajahan','kolonialisme'],
    'unity': ['persatuan','bersatu','kesatuan','gotong royong'],
    'advice': ['nasihat','petuah','bijak','pelajaran','ingatlah'],
    'international': ['internasional','dunia','global','antar bangsa'],
    'struggle': ['perjuangan','pengorbanan','berjuang','tantangan'],
    'goodbye': ['selamat tinggal','sampai jumpa','berpisah']
}

def categorize(quotes):
    cat = {t: [] for t in tags}
    for q in quotes:
        ql = q.lower()
        for t in tags:
            for kw in keywords[t]:
                if re.search(r'\b'+re.escape(kw)+r'\b', ql):
                    cat[t].append(q)
                    break
    return cat

def build_intents(cat):
    out = []
    for t, qs in cat.items():
        if not qs: continue
        inp = [
            f"berikan kutipan tentang {t}",
            f"quotes mengenai {t}",
            f"{t} quotes"
        ]
        if t=='whoami': inp += ["siapa kamu","tentang dirimu"]
        if t=='greeting': inp += ["apa kabar","selamat pagi"]
        out.append({"tag": t, "input": inp, "responses": list(dict.fromkeys(qs))})
    return out

hatta = load_quotes(hatta_csv)
soekarno = load_quotes(soekarno_csv)

data = {
    "Hatta": {"intents": build_intents(categorize(hatta))},
    "Soekarno": {"intents": build_intents(categorize(soekarno))}
}

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Saved to", output_json)


Saved to c:\PythonVSCenv\Capstone\scrapping\content_author_quotes.json


In [8]:
import pandas as pd
import json
import os
import re

base_path = r"c:\PythonVSCenv\Capstone\scrapping"
hatta_csv = os.path.join(base_path, "quotes_hatta.csv")
soekarno_csv = os.path.join(base_path, "quotes_soekarno.csv")
output_json = os.path.join(base_path, "content_by_author_and_tags.json")

def load_quotes(path):
    if not os.path.exists(path):
        return []
    df = pd.read_csv(path)
    col = 'Quotes' if 'Quotes' in df.columns else df.columns[0]
    return [q.strip().strip('"') for q in df[col].dropna().astype(str) if q.strip()]

tags = [
    'greeting','whoami','nationalism','revolution','independence',
    'unity','advice','international','struggle','goodbye'
]

keywords = {
    'greeting': ['selamat pagi','halo','hai','salam'],
    'whoami': ['aku adalah','saya adalah','identitas saya'],
    'nationalism': ['bangsa','negara','nasional','tanah air'],
    'revolution': ['revolusi','letusan','mengguncang','gempakan'],
    'independence': ['merdeka','kemerdekaan','bebas','penjajahan','kolonialisme'],
    'unity': ['persatuan','bersatu','kesatuan','gotong royong'],
    'advice': ['nasihat','petuah','bijak','pelajaran','ingatlah'],
    'international': ['internasional','dunia','global','antar bangsa'],
    'struggle': ['perjuangan','pengorbanan','berjuang','tantangan'],
    'goodbye': ['selamat tinggal','sampai jumpa','berpisah']
}

def categorize(quotes):
    cat = {t: [] for t in tags}
    for q in quotes:
        ql = q.lower()
        for t in tags:
            for kw in keywords[t]:
                if re.search(r'\b'+re.escape(kw)+r'\b', ql):
                    cat[t].append(q)
                    break
    return cat

def build_intents(cat):
    out = []
    for t, qs in cat.items():
        if not qs: continue
        inp = [
            f"berikan kutipan tentang {t}",
            f"quotes mengenai {t}",
            f"{t} quotes"
        ]
        if t=='whoami': inp += ["siapa kamu","tentang dirimu"]
        if t=='greeting': inp += ["apa kabar","selamat pagi"]
        out.append({"tag": t, "input": inp, "responses": list(dict.fromkeys(qs))})
    return out

hatta = load_quotes(hatta_csv)
soekarno = load_quotes(soekarno_csv)

data = {
    "Hatta": {"intents": build_intents(categorize(hatta))},
    "Soekarno": {"intents": build_intents(categorize(soekarno))}
}

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Saved to", output_json)

Saved to c:\PythonVSCenv\Capstone\scrapping\content_by_author_and_tags.json


#### Melakukan pergantian format ke json untuk quotes quotes content json 

In [9]:
import pandas as pd
import json
import os

# Define base path for the files
base_path = r"c:\PythonVSCenv\Capstone\scrapping"

# Define input CSV file names and output JSON file name
hatta_csv_file = 'quotes_hatta.csv'
soekarno_csv_file = 'quotes_soekarno.csv' # Pastikan file ini ada
output_json_file = 'content.json'

# Construct full paths
hatta_csv_path = os.path.join(base_path, hatta_csv_file)
soekarno_csv_path = os.path.join(base_path, soekarno_csv_file)
output_json_path = os.path.join(base_path, output_json_file)

def load_quotes_from_csv(csv_path):
    """Reads quotes from a CSV file."""
    if not os.path.exists(csv_path):
        print(f"Peringatan: File tidak ditemukan - {csv_path}")
        return []
    try:
        df = pd.read_csv(csv_path)
        # Asumsikan kolom pertama berisi kutipan, atau kolom bernama 'Quotes'
        if 'Quotes' in df.columns:
            quotes = df['Quotes'].dropna().astype(str).tolist()
        elif not df.empty:
            quotes = df.iloc[:, 0].dropna().astype(str).tolist()
        else:
            quotes = []
        
        # Membersihkan kutipan dari tanda kutip ganda yang mungkin ada di awal/akhir dari CSV
        cleaned_quotes = [q.strip('"') for q in quotes]
        return cleaned_quotes
    except Exception as e:
        print(f"Error saat membaca {csv_path}: {e}")
        return []

# Load quotes
hatta_quotes = load_quotes_from_csv(hatta_csv_path)
soekarno_quotes = load_quotes_from_csv(soekarno_csv_path) # Pastikan file quotes_soekarno.csv ada

# Initialize intents list
intents_list = []

# Create intent for Hatta's quotes
if hatta_quotes:
    hatta_intent = {
        "tag": "quote_hatta",
        "input": [
            "kutipan hatta", "quotes hatta", "kata bijak hatta", 
            "pemikiran hatta", "nasihat hatta"
        ],
        "responses": hatta_quotes
    }
    intents_list.append(hatta_intent)
else:
    print(f"Tidak ada kutipan yang dimuat dari {hatta_csv_path}")

# Create intent for Soekarno's quotes
if soekarno_quotes:
    soekarno_intent = {
        "tag": "quote_soekarno",
        "input": [
            "kutipan soekarno", "quotes soekarno", "kata bijak soekarno", 
            "pemikiran soekarno", "pidato soekarno", "nasihat soekarno"
        ],
        "responses": soekarno_quotes
    }
    intents_list.append(soekarno_intent)
else:
    print(f"Tidak ada kutipan yang dimuat dari {soekarno_csv_path}. Pastikan file tersebut ada dan formatnya benar.")

# Final JSON structure
final_json_data = {
    "intents": intents_list
}

# Write to JSON file
try:
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(final_json_data, f, ensure_ascii=False, indent=4)
    print(f"Berhasil mengonversi kutipan ke JSON: {output_json_path}")
    if not intents_list:
        print("Peringatan: File JSON yang dihasilkan kosong karena tidak ada kutipan yang ditemukan.")
except Exception as e:
    print(f"Error saat menulis file JSON: {e}")


import pandas as pd
import json
import os

# Define base path for the files
base_path = r"c:\PythonVSCenv\Capstone\scrapping"

# Define input CSV file names and output JSON file name
hatta_csv_file = 'quotes_hatta.csv'
soekarno_csv_file = 'quotes_soekarno.csv' # Pastikan file ini ada
output_json_file = 'content.json'

# Construct full paths
hatta_csv_path = os.path.join(base_path, hatta_csv_file)
soekarno_csv_path = os.path.join(base_path, soekarno_csv_file)
output_json_path = os.path.join(base_path, output_json_file)

def load_quotes_from_csv(csv_path):
    """Reads quotes from a CSV file."""
    if not os.path.exists(csv_path):
        print(f"Peringatan: File tidak ditemukan - {csv_path}")
        return []
    try:
        df = pd.read_csv(csv_path)
        # Asumsikan kolom pertama berisi kutipan, atau kolom bernama 'Quotes'
        if 'Quotes' in df.columns:
            quotes = df['Quotes'].dropna().astype(str).tolist()
        elif not df.empty:
            quotes = df.iloc[:, 0].dropna().astype(str).tolist()
        else:
            quotes = []
        
        # Membersihkan kutipan dari tanda kutip ganda yang mungkin ada di awal/akhir dari CSV
        cleaned_quotes = [q.strip('"') for q in quotes]
        return cleaned_quotes
    except Exception as e:
        print(f"Error saat membaca {csv_path}: {e}")
        return []

# Load quotes
hatta_quotes = load_quotes_from_csv(hatta_csv_path)
soekarno_quotes = load_quotes_from_csv(soekarno_csv_path) # Pastikan file quotes_soekarno.csv ada

# Initialize intents list
intents_list = []

# Create intent for Hatta's quotes
if hatta_quotes:
    hatta_intent = {
        "tag": "quote_hatta",
        "input": [
            "kutipan hatta", "quotes hatta", "kata bijak hatta", 
            "pemikiran hatta", "nasihat hatta"
        ],
        "responses": hatta_quotes
    }
    intents_list.append(hatta_intent)
else:
    print(f"Tidak ada kutipan yang dimuat dari {hatta_csv_path}")

# Create intent for Soekarno's quotes
if soekarno_quotes:
    soekarno_intent = {
        "tag": "quote_soekarno",
        "input": [
            "kutipan soekarno", "quotes soekarno", "kata bijak soekarno", 
            "pemikiran soekarno", "pidato soekarno", "nasihat soekarno"
        ],
        "responses": soekarno_quotes
    }
    intents_list.append(soekarno_intent)
else:
    print(f"Tidak ada kutipan yang dimuat dari {soekarno_csv_path}. Pastikan file tersebut ada dan formatnya benar.")

# Final JSON structure
final_json_data = {
    "intents": intents_list
}

# Write to JSON file
try:
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(final_json_data, f, ensure_ascii=False, indent=4)
    print(f"Berhasil mengonversi kutipan ke JSON: {output_json_path}")
    if not intents_list:
        print("Peringatan: File JSON yang dihasilkan kosong karena tidak ada kutipan yang ditemukan.")
except Exception as e:
    print(f"Error saat menulis file JSON: {e}")

Berhasil mengonversi kutipan ke JSON: c:\PythonVSCenv\Capstone\scrapping\content.json
Berhasil mengonversi kutipan ke JSON: c:\PythonVSCenv\Capstone\scrapping\content.json


#### Buat Input Form Manual untuk ditambahkan ke json

In [2]:
import requests
import json

# Data JSON yang akan dikirim (diperbarui)
data_to_send = {
    "Hatta": {
        "intents": [
            {
                "tag": "nationalism",
                "input": [
                    "berikan kutipan tentang nationalism",
                    "quotes mengenai nationalism",
                    "nationalism quotes",
                    "nasionalisme",
                    "quotes nasionalisme"
                ],
                "responses": [
                    "Hanya ada satu negara yang pantas menjadi negaraku. Ia tumbuh dengan perbuatan dan perbuatan itu adalah perbuatanku."
                ]
            },
            {
                "tag": "revolution",
                "input": [
                    "berikan kutipan tentang revolution",
                    "quotes mengenai revolution",
                    "revolution quotes",
                    "apakah revolusi kita sudah selesai"
                ],
                "responses": [
                    "Salah besar orang yang mengatakan revolusi kita belum selesai. Revolusi adalah letusan masyarakat sekonyong-konyong yang melaksanakanUmwertung aller Werte(penilaian kembali atas semua nilai). Revolusi mengguncang lantai dan sendi; pasak dan tiang longgar semuanya. Sebab itu saat revolusi tidak dapat berlaku terlalu lama, tidak lebih dari beberapa minggu, atau beberapa bulan. Sesudah itu harus dibendung, datang masa konsolidasi untuk merealisasi hasil daripada revolusi itu. Yang belum selesai bukanlah revolusi itu, melainkan usaha menyelenggarakan cita-citanya di dalam waktu, setelah fundamen dibentangkan."
                ]
            },
            {
                "tag": "independence",
                "input": [
                    "berikan kutipan tentang independence",
                    "quotes mengenai independence",
                    "independence quotes"
                ],
                "responses": [
                    "Aku rela di penjara asalkan bersama buku, karena dengan buku aku bebas.",
                    "Partai adalah alat publik untuk menyampaikan aspirasi politiknya karena itu sejauh tetap memperjuangkan kemerdekaan dan menjamin keamanan, pemerintah menyukai timbulnya partai-partai politik, karena dengan adanya partai-partai itulah dipimpin ke jalan yang teratur segala aliran paham yang ada di masyarakat."
                ]
            },
            {
                "tag": "partai",
                "input": [
                    "apa itu partai",
                    "partai",
                    "partai politik"
                ],
                "responses": [
                    "Partai adalah alat publik untuk menyampaikan aspirasi politiknya karena itu sejauh tetap memperjuangkan kemerdekaan dan menjamin keamanan, pemerintah menyukai timbulnya partai-partai politik, karena dengan adanya partai-partai itulah dipimpin ke jalan yang teratur segala aliran paham yang ada di masyarakat."
                ]
            },
            {
                "tag": "unity",
                "input": [
                    "berikan kutipan tentang unity",
                    "quotes mengenai unity",
                    "unity quotes"
                ],
                "responses": [
                    "Apa yang dikatakan persatuan (oleh Soekarno) sebenarnya tak lain dari per-sate-an. Daging kerbau, daging sapi, dan daging kambing disate jadi satu. Persatuan segala golongan ini sama artinya dengan mengorbankan asas masing-masing."
                ]
            },
            {
                "tag": "international",
                "input": [
                    "berikan kutipan tentang international",
                    "quotes mengenai international",
                    "international quotes",
                    "dunia",
                    "dunia internasional"
                ],
                "responses": [
                    "Siapa yang hidup dalam dunia pikiran, dapat melepaskan dirinya daripada gangguan hidup sehari-hari",
                    "Beri aku 10 pemuda, niscaya akan kuguncangkan dunia.",
                    "Dunia ini adalah satu kesatuan, dan tidak ada satu bangsa pun yang dapat hidup sendiri. Kita harus saling membantu, saling mengerti, dan saling menghormati."
                ]
            },
            {
                "tag": "Soekarno",
                "input": [
                    "kutipan hatta tentang soekarno",
                    "hatta dan soekarno",
                    "kritik hatta terhadap soekarno",
                    "kenapa anda berhenti bekerja sama dengan soekarno"
                ],
                "responses": [
                    "Saya tidak dapat bekerja sama dengan Presiden Soekarno lagi, karena beliau tidak mau dibatasi oleh konstitusi.",
                    "Saya tidak dapat menyetujui sistem demokrasi terpimpin sebagaimana yang dijalankan oleh Presiden Soekarno. Demokrasi terpimpin itu pada hakekatnya adalah kediktatoran yang dibungkus dengan selubung demokrasi. Dalam sistem itu, rakyat tidak lagi mempunyai kedaulatan, sebab kedaulatan itu telah berpindah ke tangan pemimpin."
                ]
            },
            {
                "tag": "demokrasi",
                "input": [
                    "kutipan hatta tentang demokrasi",
                    "pendapat hatta tentang demokrasi",
                    "demokrasi menurut hatta"
                ],
                "responses": [
                    "Akhir-akhir ini, salam kemerdekaan mulai kita lupakan, karena kita telah melupakan makna sejati dari kemerdekaan itu sendiri. Salam tersebut perlahan menghilang karena kenyataannya, kita telah menyimpang dari apa yang kita proklamasikan. Oleh karena itu, kita harus menyelamatkan rakyat dari kekuasaan diktator. Sebab, jika ada diktator yang berhasil memakmurkan rakyat, maka kemakmuran itu hanya terbatas selama masa kekuasaan diktator tersebut.",
                    "SEJARAH Indonesia sejak sepuluh tahun terakhir ini banyak memperlihatkan pertentangan antara idealisme dan realita. Idealisme, yang bertujuan menciptakan suatu pemerintahan yang adil, yang akan melaksanakan demokrasi yang sebaik-baiknya, dan kemakmuran yang sebesar-besarnya. Sementara realita saat ini, pemerintah yang dalam perkembangannya kelihatan semakin jauh dari demokrasi yang sebenarnya."
                ]
            },
            {
                "tag": "negara",
                "input": [
                    "kutipan hatta tentang negara",
                    "negara menurut hatta",
                    "pembangunan negara menurut hatta"
                ],
                "responses": [
                    "Kemajuan dari suatu negara adalah kemajuan pendidikan dari sekolah dan kemajuan kesehatan dari rumah sakit."
                ]
            }
        ]
    },
    "Soekarno": {
        "intents": [
            {
                "tag": "nationalism",
                "input": [
                    "berikan kutipan tentang nationalism",
                    "quotes mengenai nationalism",
                    "nationalism quotes",
                    "apa itu bangsa",
                    "mengapa bangsa kita besar",
                    "mengapa kita harus nasionalis",
                    "mengapa kita harus mencintai bangsa",
                    "nasionalisme"
                ],
                "responses": [
                    "Kami menggoyangkan langit, menggempakan darat, dan menggelorakan samudera agar tidak jadi bangsa yang hidup hanya dari 2 ½ sen sehari. Bangsa yang kerja keras, bukan bangsa tempe, bukan bangsa kuli. Bangsa yang rela menderita demi pembelian cita-cita",
                    "Bangsa yang besar adalah bangsa yang menghargai jasa pahlawannya.",
                    "Dalam tahun 1882 Ernest Renan telah membuka pendapatnja tentang faham \"bangsa\" itu. \"Bangsa\" itu menurut pudjangga ini ada suatu nyawa, suatu asas-akal, yang terjadi dari dua hal: pertama-tama rakyat itu dulunya harus bersama-sama menjalani suatu riwayat; kedua rakyat itu sekarang harus mempunyai kemauan, keinginan hidup menjadi satu. Bukannya jenis (ras), bukannya pula batas-batas negeri yang menjadikan \"bangsa\" itu.",
                    "Nasionalisme itu jalah suatu itikad; suatu keinsyafan rakjat bahwa rakjat itu ada satu golongan, satu \"bangsa\"!",
                    "Nasionalis yang sejati, yang nasionalismenya itu bukan timbul semata-mata tiruan dari nasionalisme barat akan tetapi timbul dari rasa cinta akan manusia dan kemanusiaan"
               ]
            },
            {
                "tag": "bangsa",
                "input": [
                    "apa itu bangsa",
                    "bangsa"
                ],
                "responses": [
                    "Bangsa yang besar adalah bangsa yang menghargai jasa pahlawannya.",
                    "Dalam tahun 1882 Ernest Renan telah membuka pendapatnja tentang paham \"bangsa\" itu. \"Bangsa\" itu menurut pudjangga ini ada suatu njawa, suatu azas-akal, jang terdjadi dari dua hal: pertama-tama rakjat itu dulunja harus bersama-sama mendjalani suatu riwayat; kedua rakjat itu sekarang harus mempunjai kemauan, keinginan hidup mendjadi satu. Bukannja djenis (ras), bukannja pula batas-batas negeri jang mendjadikan \"bangsa\" itu.",
                    "Bangsa itu adalah suatu persatuan perangai yang terjadi dari persatuan hal-ikhwal yang telah dijalani oleh rakyat itu."
            ]
            },
            {
                "tag": "ideologies",
                "input": [
                    "kutipan soekarno tentang ideologi",
                    "ideologi menurut soekarno",
                    "pendapat soekarno tentang ideologi"
                ],
                "responses": [
                    "Saya menerima semua ideologi selama ideologi tersebut tidak merusak Indonesia"
                ]
            },

            {
                "tag": "revolution",
                "input": [
                    "berikan kutipan tentang revolution",
                    "quotes mengenai revolution",
                    "revolution quotes"
                ],
                "responses": [
                    "Wanita Indonesia, kewajibanmu telah terang! Sekarang, ikutilah serta mutlak dalam usaha menyelamatkan Republik, dan nanti jika Republik telah selamat, ikutilah serta mutlak dalam usaha menyusun Negara Nasional. Jangan ketinggalan didalam Revolusi Nasional ini dai awal sampai akhirnya, dan jangan ketinggalan pula nanti di dalam usaha menyusun masyarakat keadilan sosial dan kesejahteraan sosial. Didalam masyarakat keadilan sosial dan kesejahteraan sosial telah engkau nanti menjadi wanita yang bahagia, wanita yang Merdeka!\"",
                    "Bahwa yang menyebabkan kolonisasi itu bukanlah keinginan pada kemasyhuran, bukan keinginan melihat dunia-asing, bukan keinginan merdeka, dan bukan pula oleh karena negeri rakyat yang menjalankan kolonisasi itu ada terlampau sesak oleh banyaknya penduduk, sebagai yang telah diajarkan oleh Gustav Klemm, akan tetapi asalnya kolonisasi ialah teristimewa soal rezeki."

                ]
            },
            {
                "tag": "independence",
                "input": [
                    "berikan kutipan tentang independence",
                    "quotes mengenai independence",
                    "independence quotes",
                    "kemerdekaan",
                    "merdeka"
                ],
                "responses": [
                    "Wanita Indonesia, kewajibanmu telah terang! Sekarang, ikutilah serta mutlak dalam usaha menyelamatkan Republik, dan nanti jika Republik telah selamat, ikutilah serta mutlak dalam usaha menyusun Negara Nasional. Jangan ketinggalan didalam Revolusi Nasional ini dai awal sampai akhirnya, dan jangan ketinggalan pula nanti di dalam usaha menyusun masyarakat keadilan sosial dan kesejahteraan sosial. Didalam masyarakat keadilan sosial dan kesejahteraan sosial telah engkau nanti menjadi wanita yang bahagia, wanita yang Merdeka!\"",
                    "Juga diatas pundak wanitalah terletak kewajiban untuk tidak ketinggalan di dalam perjuangan ini, dalam mana diperjuangkan kemerdekaan mereka dan pembebasan mereka. Mereka sendirilah harus membuktikan, bahwa mereka mengerti benar-benar tempat mereka dalam perjuangan sekarang yang mengejar masa depan yang lebih baik itu\"",
                    "Kemerdekaan ialah hak segala bangsa, dan oleh sebab itu penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan peri kemanusiaan dan peri keadilan."
                ]
            },
            {
                "tag": "kolonialisme",
                "input": [
                    "kutipan soekarno tentang kolonialisme",
                    "kolonialisme menurut soekarno"
                ],
                "responses": [
                    "Yang menyebabkan kolonisasi bukanlah karena keinginan untuk mencari ketenaran, bukan karena ingin melihat dunia asing, bukan karena dorongan untuk merdeka, dan juga bukan karena negeri rakyat yang melakukan kolonisasi itu terlalu padat penduduknya. Tetapi pada dasarnya, kolonisasi terutama berakar pada persoalan mencari penghidupan"
                ]
            },
            {
                "tag": "unity",
                "input": [
                    "berikan kutipan tentang unity",
                    "quotes mengenai unity",
                    "unity quotes",
                    "persatuan"
                ],
                "responses": [
                    "Bangsa itu adalah suatu persatuan perangai yang terjadi dari persatuan hal-ikhwal yang telah dijalani oleh rakyat itu."
                ]
            },
            {
                "tag": "international",
                "input": [
                    "berikan kutipan tentang international",
                    "quotes mengenai international",
                    "international quotes"
                ],
                "responses": [
                    "Yang menyebabkan kolonisasi bukanlah karena keinginan untuk mencari ketenaran, bukan karena ingin melihat dunia asing, bukan karena dorongan untuk merdeka, dan juga bukan karena negeri rakyat yang melakukan kolonisasi itu terlalu padat penduduknya. Tetapi pada dasarnya, kolonisasi terutama berakar pada persoalan mencari penghidupan"
                ]
            },
            {
                "tag": "struggle",
                "input": [
                    "berikan kutipan tentang struggle",
                    "quotes mengenai struggle",
                    "struggle quotes",
                    "perjuangan",
                    "perjuangan bangsa",
                    "perjuangan kemerdekaan",
                    "kesulitan",
                    "kesulitan bangsa"
                ],
                "responses": [
                    "Juga diatas pundak wanitalah terletak kewajiban untuk tidak ketinggalan di dalam perjuangan ini, dalam mana diperjuangkan kemerdekaan mereka dan pembebasan mereka. Mereka sendirilah harus membuktikan, bahwa mereka mengerti benar-benar tempat mereka dalam perjuangan sekarang yang mengejar masa depan yang lebih baik itu\" ",
                    "Dan kamu, kaum wanita Indonesia,-akhirnya nasibmu adalah di tangan kamu sendiri. Saya memberi peringatan kepada kaum laku-laki itu untuk memberi keyakinan kepada mereka tentang hargamu dalam perjuangan, tetapi kamu sendiri harus menjadi sadar, kamu sendiri harus terjun mutlak dalam perjuangan\" ",
                     "Belajar tanpa berpikir itu tidak berguna, tetapi berpikir tanpa belajar sangat berbahaya!",
                     "Perjuanganku lebih mudah karena mengusir penjajah, tapi perjuanganmu akan lebih sulit karena melawan bangsamu sendiri."
                ]
            }
        ]
    }
}

# URL endpoint API (tetap sama)
api_url = "https://capstone-five-dusky.vercel.app/chatbot/tags"

# Header untuk request (tetap sama)
headers = {
    "Content-Type": "application/json"
}

# Melakukan POST request
try:
    response = requests.post(api_url, data=json.dumps(data_to_send), headers=headers)

    # Mengecek status code response
    if response.status_code == 200 or response.status_code == 201: # 201 biasanya berarti 'Created'
        print("Data berhasil dikirim!")
        try:
            print("Response JSON:", response.json())
        except json.JSONDecodeError:
            print("Server tidak mengembalikan JSON yang valid. Response text:", response.text)
    else:
        print(f"Gagal mengirim data. Status code: {response.status_code}")
        print("Response text:", response.text)

except requests.exceptions.RequestException as e:
    print(f"Terjadi kesalahan saat mengirim request: {e}")

Gagal mengirim data. Status code: 400
Response text: {"statusCode":400,"error":"Bad Request","message":"Invalid request payload input"}
